# Imports

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.constraints import max_norm

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

# Read

In [4]:
df=pd.read_csv('soccer.csv')
df=df[~df['Match'].isna()]
df['Team1']=df['Team1']#.astype(str)
df['Team2']=df['Team2']#.astype(str)
df['Result']=df['Result']*0.5

d_count=df.groupby('Match').count()
d_ratio=(d_count['Team1']/d_count['Team2']).values
df['Playerno']=0
for i in range(len(d_count)):
    df.loc[df['Match']==d_count.index[i],'Playerno']=d_ratio[i]
df_r=df.copy()
df_r['Team1']=df['Team2']
df_r['Team2']=df['Team1']
df_r['Result']=1-df['Result']
df_r['Playerno']=1/df['Playerno']
df_r['Match']=df['Match']+df['Match'].max()
df=pd.concat([df,df_r])
df1=pd.concat([df,pd.get_dummies(df['Team1'])],axis=1)
df1=df1.reset_index(drop=True)
for i in df1.columns[4:]:
    df1.loc[df1['Team2']==i,i]= 0.5
df1=df1.groupby('Match',as_index=False).max()
for i in df1.columns[2:]:
    df1.loc[df1[i]==0.5,i]=-1
df1

<ipython-input-4-e475a5d645d4>:23: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  df1=df1.groupby('Match',as_index=False).max()


,Match,Result,Playerno,_Afshin,_Ali Amiri,_Ali Asadinia,_Ali Izadi,_Ali Kalantari,_Alireza Najafi,_Alireza Nazeri,...,_Mohammad_Amin Ghiasi,_Mohsen Mirzaei,_Payam Zh. Monfared,_Pedram Ensandoost,_Saeid Hobbi,_Saleh Mohammad Rezaei,_Shayan,_Sina Eslami,_Sina Nozarian,_Yousef Nikzad
0,1,0.571429,1.000000,-1.0,0.0,-1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,2,0.500000,1.000000,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0
2,3,0.550000,1.200000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0
3,4,0.550000,1.000000,-1.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,0.600000,1.200000,-1.0,0.0,1.0,-1.0,-1.0,1.0,-1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0.585000,1.000000,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,...,0.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,1.0,-1.0
96,97,0.475000,1.000000,0.0,1.0,0.0,0.0,-1.0,-1.0,0.0,...,1.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,1.0
97,98,0.000000,1.000000,-1.0,1.0,-1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,-1.0,0.0
98,99,0.500000,0.833333,1.0,1.0,1.0,1.0,0.0,-1.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0


# Mean win score

In [5]:
topz=[]
dic1={}
for i in df1.columns[3:]:
    dic1[i]=df1[df1[i]==1]['Result'].mean()-0.5
Rating=dict(sorted(dic1.items(), key=lambda item: item[1]))
Rating

{'_Saleh Mohammad Rezaei': -0.25,
 '_Ali Kalantari': -0.13354166665625,
 '_Behnam Hajimosaei': -0.08758095238000002,
 '_Alireza Nazeri': -0.06469187676470584,
 '_Alireza Najafi': -0.03917919798684205,
 '_Ali Asadinia': -0.03840852130263156,
 '_Yousef Nikzad': -0.02818181818181814,
 '_Ali Izadi': -0.02262626262121209,
 '_Alireza Ranjbar': -0.017817460321428502,
 '_Ashkan': -0.01749999999999996,
 '_Payam Zh. Monfared': -0.017112135177419274,
 '_Afshin': -0.008494897964285708,
 '_Hadi': -0.005912698416666695,
 '_Mohammad_Amin Ghiasi': -0.003761904749999989,
 '_Amin': 0.010714285714285676,
 '_Jamshid': 0.011038961045454543,
 '_Erfan Moeini': 0.01666666666666672,
 '_Sina Nozarian': 0.01904118404054056,
 '_Milad': 0.037056277045454555,
 '_Sina Eslami': 0.0643087557580645,
 '_Mehdi Saadati': 0.07277777777777772,
 '_Shayan': 0.07374999999999998,
 '_Pedram Ensandoost': 0.07794117647058818,
 '_Ali Amiri': 0.07970238095833337,
 '_Saeid Hobbi': 0.10733333329999994,
 '_Maghsad': 0.16666666666666663

# Removing Outliers

Let's see which player has the highest played game

In [6]:
dic2={}
for i in df1.columns[3:]:
    dic2[i]=int(df1[df1[i]==1][i].sum())
dic2=dict(sorted(dic2.items(), key=lambda item: item[1]))
dic2

{'_Maghsad': 3,
 '_Saeid Hobbi': 5,
 '_Saleh Mohammad Rezaei': 5,
 '_Erfan Moeini': 6,
 '_Shayan': 8,
 '_Jamshid': 11,
 '_Ali Amiri': 12,
 '_Ali Kalantari': 16,
 '_Alireza Nazeri': 17,
 '_Pedram Ensandoost': 17,
 '_Hadi': 18,
 '_Ashkan': 20,
 '_Mohammad_Amin Ghiasi': 20,
 '_Amin': 21,
 '_Milad': 22,
 '_Yousef Nikzad': 22,
 '_Behnam Hajimosaei': 25,
 '_Mohsen Mirzaei': 25,
 '_Mehdi Saadati': 27,
 '_Afshin': 28,
 '_Payam Zh. Monfared': 31,
 '_Sina Eslami': 31,
 '_Ali Izadi': 33,
 '_Sina Nozarian': 37,
 '_Ali Asadinia': 38,
 '_Alireza Najafi': 38,
 '_Alireza Ranjbar': 42}

After talking to Sina, We decided to remove the ones with less than 8 games

In [7]:
removables=[]
for i,j in dic2.items():
    if j<8:
        removables.append(i)
df1=df1.drop(columns=removables)
df1

,Match,Result,Playerno,_Afshin,_Ali Amiri,_Ali Asadinia,_Ali Izadi,_Ali Kalantari,_Alireza Najafi,_Alireza Nazeri,...,_Mehdi Saadati,_Milad,_Mohammad_Amin Ghiasi,_Mohsen Mirzaei,_Payam Zh. Monfared,_Pedram Ensandoost,_Shayan,_Sina Eslami,_Sina Nozarian,_Yousef Nikzad
0,1,0.571429,1.000000,-1.0,0.0,-1.0,0.0,0.0,1.0,0.0,...,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,2,0.500000,1.000000,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0
2,3,0.550000,1.200000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,-1.0,1.0,1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,1.0
3,4,0.550000,1.000000,-1.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,5,0.600000,1.200000,-1.0,0.0,1.0,-1.0,-1.0,1.0,-1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0.585000,1.000000,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,...,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0
96,97,0.475000,1.000000,0.0,1.0,0.0,0.0,-1.0,-1.0,0.0,...,0.0,1.0,1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,1.0
97,98,0.000000,1.000000,-1.0,1.0,-1.0,0.0,1.0,1.0,1.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0
98,99,0.500000,0.833333,1.0,1.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0


### Mean Score now:

In [8]:
topz=[]
dic1={}
for i in df1.columns[3:]:
    dic1[i]=df1[df1[i]==1]['Result'].mean()-0.5
Rating=dict(sorted(dic1.items(), key=lambda item: item[1]))
Rating

{'_Ali Kalantari': -0.13354166665625,
 '_Behnam Hajimosaei': -0.08758095238000002,
 '_Alireza Nazeri': -0.06469187676470584,
 '_Alireza Najafi': -0.03917919798684205,
 '_Ali Asadinia': -0.03840852130263156,
 '_Yousef Nikzad': -0.02818181818181814,
 '_Ali Izadi': -0.02262626262121209,
 '_Alireza Ranjbar': -0.017817460321428502,
 '_Ashkan': -0.01749999999999996,
 '_Payam Zh. Monfared': -0.017112135177419274,
 '_Afshin': -0.008494897964285708,
 '_Hadi': -0.005912698416666695,
 '_Mohammad_Amin Ghiasi': -0.003761904749999989,
 '_Amin': 0.010714285714285676,
 '_Jamshid': 0.011038961045454543,
 '_Sina Nozarian': 0.01904118404054056,
 '_Milad': 0.037056277045454555,
 '_Sina Eslami': 0.0643087557580645,
 '_Mehdi Saadati': 0.07277777777777772,
 '_Shayan': 0.07374999999999998,
 '_Pedram Ensandoost': 0.07794117647058818,
 '_Ali Amiri': 0.07970238095833337,
 '_Mohsen Mirzaei': 0.17305714285999996}

### Looks Good

In [9]:
list_of_players=df1.columns[3:]

In [10]:
list_of_players

Index(['_Afshin', '_Ali Amiri', '_Ali Asadinia', '_Ali Izadi',
       '_Ali Kalantari', '_Alireza Najafi', '_Alireza Nazeri',
       '_Alireza Ranjbar', '_Amin', '_Ashkan', '_Behnam Hajimosaei', '_Hadi',
       '_Jamshid', '_Mehdi Saadati', '_Milad', '_Mohammad_Amin Ghiasi',
       '_Mohsen Mirzaei', '_Payam Zh. Monfared', '_Pedram Ensandoost',
       '_Shayan', '_Sina Eslami', '_Sina Nozarian', '_Yousef Nikzad'],
      dtype='object')

# Pairs Analysis

### Get every pair:

In [11]:
pair_of_players=[]
for c,i in enumerate(list_of_players):
    for x in range(len(list_of_players)):
        if x>c:
            pair_of_players.append([i,list_of_players[x]])
    

In [12]:
pair_of_players[-1]

['_Sina Nozarian', '_Yousef Nikzad']

### Extract stats for every pair:

In [13]:
def get_pair_winrate(a,b,df1):
    mean=df1[(df1[a]==1)&(df1[b]==1)]['Result'].mean()
    count=df1[(df1[a]==1)&(df1[b]==1)]['Result'].count()
    median=df1[(df1[a]==1)&(df1[b]==1)]['Result'].median()
    return mean,median,count

In [14]:
dr=pd.DataFrame()
for i in pair_of_players:
    mean,median,count=get_pair_winrate(i[0],i[1],df1)
    dr=dr.append({'player1':i[0],'player2':i[1],'mean':mean,'median':median,'count':count},ignore_index=True)
    dr=dr.dropna(how='any')
dr

,player1,player2,mean,median,count
0,_Afshin,_Ali Amiri,0.495000,0.495000,2.0
1,_Afshin,_Ali Asadinia,0.478143,0.450000,15.0
2,_Afshin,_Ali Izadi,0.490625,0.500000,16.0
3,_Afshin,_Ali Kalantari,0.321000,0.400000,5.0
4,_Afshin,_Alireza Najafi,0.336250,0.172500,4.0
...,...,...,...,...,...
228,_Shayan,_Sina Nozarian,0.558333,0.525000,3.0
229,_Shayan,_Yousef Nikzad,0.432500,0.432500,2.0
230,_Sina Eslami,_Sina Nozarian,0.587500,0.587500,2.0
231,_Sina Eslami,_Yousef Nikzad,0.417857,0.428571,3.0


Before we start, let's remove the ones with low counts, or make a new parameter that scores based on number of games

## Setting Score

In [15]:
dr['pair_score']=((dr['median']-0.5)*10+(dr['mean']-0.5)*10)*(np.log(dr['count']+15)-2.83)**1

In [16]:
dr['pair_score'].describe()

count    233.000000
mean      -0.004195
std        0.578186
min       -2.139185
25%       -0.203377
50%        0.000000
75%        0.191371
max        2.877068
Name: pair_score, dtype: float64

# Most frequent pairs:

In [17]:
dr.sort_values('count',ascending=False).head(10)

,player1,player2,mean,median,count,pair_score
57,_Ali Asadinia,_Sina Nozarian,0.507800,0.500000,25.0,0.066993
45,_Ali Asadinia,_Alireza Ranjbar,0.491071,0.500000,22.0,-0.069725
133,_Alireza Ranjbar,_Sina Nozarian,0.539655,0.500000,20.0,0.287635
215,_Mohsen Mirzaei,_Sina Eslami,0.663158,0.750000,19.0,2.877068
121,_Alireza Ranjbar,_Ashkan,0.509375,0.500000,16.0,0.056624
2,_Afshin,_Ali Izadi,0.490625,0.500000,16.0,-0.056624
129,_Alireza Ranjbar,_Payam Zh. Monfared,0.438943,0.500000,16.0,-0.368774
41,_Ali Asadinia,_Ali Izadi,0.358958,0.316667,16.0,-1.959183
18,_Afshin,_Sina Eslami,0.531238,0.500000,15.0,0.178431
1,_Afshin,_Ali Asadinia,0.478143,0.450000,15.0,-0.410446


# Highest pair scores

In [18]:

dr=dr.sort_values('pair_score',ascending=False)
dr=dr.reset_index(drop=True)
dr.head(15)

,player1,player2,mean,median,count,pair_score
0,_Mohsen Mirzaei,_Sina Eslami,0.663158,0.750000,19.0,2.877068
1,_Pedram Ensandoost,_Sina Eslami,0.697273,0.750000,11.0,1.914759
2,_Alireza Najafi,_Mehdi Saadati,0.616369,0.750000,12.0,1.706682
3,_Alireza Najafi,_Mohsen Mirzaei,0.642857,0.750000,11.0,1.681808
4,_Mehdi Saadati,_Pedram Ensandoost,0.722222,0.750000,9.0,1.643588
5,_Alireza Najafi,_Pedram Ensandoost,0.628636,0.750000,11.0,1.620929
6,_Mohsen Mirzaei,_Pedram Ensandoost,0.650000,0.750000,10.0,1.555503
7,_Mehdi Saadati,_Mohsen Mirzaei,0.666667,0.750000,9.0,1.450224
8,_Milad,_Sina Nozarian,0.628788,0.666667,11.0,1.264831
9,_Ali Amiri,_Mohsen Mirzaei,0.708571,0.840000,6.0,1.176809


In [19]:
dr[(dr['player1']=='_Milad')|(dr['player2']=='_Milad')]['count'].sum()

113.0

# Least Pair reliant players

In [20]:
np.log(43)

3.7612001156935624

In [21]:
dp=(dr.groupby('player1')['pair_score'].agg(['max','min'])+dr.groupby('player2')['pair_score'].agg(['max','min']))/2
#Adding count:
dp['count']=0
for i in dp.index.values:
    count=dr[(dr['player1']==i)|(dr['player2']==i)]['count'].sum()
    dp.loc[dp.index.values==i,'count']=count

dp.loc['_Afshin','min']=dr.groupby('player1')['pair_score'].agg(['max','min'])['min'][0]
dp.loc['_Afshin','max']=dr.groupby('player1')['pair_score'].agg(['max','min'])['max'][0]

dp.loc['_Yousef Nikzad','min']=dr.groupby('player2')['pair_score'].agg(['max','min'])['min'][-1]
dp.loc['_Yousef Nikzad','max']=dr.groupby('player2')['pair_score'].agg(['max','min'])['max'][-1]
dp['reliant_factor']=((dp['max']-5*dp['min'])-(np.log(dp['count']))*0.4)/10
dp=dp.sort_values('reliant_factor')
dp

,max,min,count,reliant_factor
_Milad,1.146762,-0.150794,113,0.000977
_Mehdi Saadati,1.675135,-0.069339,119,0.011018
_Shayan,0.170349,-0.289225,43,0.011200
_Ali Amiri,0.588244,-0.287217,63,0.036708
_Sina Nozarian,0.452864,-0.500374,171,0.089807
_Mohammad_Amin Ghiasi,0.481613,-0.452898,98,0.091212
_Hadi,0.386972,-0.481783,98,0.096190
_Jamshid,0.067129,-0.510517,50,0.105491
_Ashkan,0.383738,-0.503838,82,0.114024
_Amin,0.432885,-0.588670,93,0.156319


# Best pair for each player:

In [22]:
dr1=dr[dr['count']>2]
colz=['Player','Best_Match','Worst_Match']
colz.extend(dr.columns[2:].tolist())
dp2=pd.DataFrame(columns=colz)
dp2
for i in list_of_players:
    #Max:
    d_temp=dr1[(dr1['player1']==i)|(dr1['player2']==i)].values[0]
    first_name=d_temp[0]
    second_name=d_temp[1]
    if first_name!=i:
              Best_name=first_name
    else:
              Best_name=second_name
    #Min:
    d_temp_2=dr1[(dr1['player1']==i)|(dr1['player2']==i)].values[-1]
    first_name_worst=d_temp_2[0]
    second_name_worst=d_temp_2[1]
    if first_name_worst!=i:
              Worst_name=first_name_worst
    else:
              Worst_name=second_name_worst
    
    
    dp2=dp2.append({'Player':i,'Best_Match':Best_name,'Worst_Match':np.nan,'mean':d_temp[2],'median':d_temp[3],'count':d_temp[4],'pair_score':d_temp[5]},ignore_index=True)
    dp2=dp2.append({'Player':i,'Best_Match':np.nan,'Worst_Match':Worst_name,'mean':d_temp_2[2],'median':d_temp_2[3],'count':d_temp_2[4],'pair_score':d_temp_2[5]},ignore_index=True)
dp2

,Player,Best_Match,Worst_Match,mean,median,count,pair_score
0,_Afshin,_Sina Nozarian,NaN,0.601429,0.585000,7.0,0.486658
1,_Afshin,NaN,_Payam Zh. Monfared,0.298000,0.000000,5.0,-1.163441
2,_Ali Amiri,_Mohsen Mirzaei,NaN,0.708571,0.840000,6.0,1.176809
3,_Ali Amiri,NaN,_Behnam Hajimosaei,0.396000,0.490000,5.0,-0.188935
4,_Ali Asadinia,_Milad,NaN,0.539196,0.567500,8.0,0.325951
5,_Ali Asadinia,NaN,_Ali Izadi,0.358958,0.316667,16.0,-1.959183
6,_Ali Izadi,_Mohsen Mirzaei,NaN,0.640500,0.625000,10.0,1.032465
7,_Ali Izadi,NaN,_Ali Asadinia,0.358958,0.316667,16.0,-1.959183
8,_Ali Kalantari,_Sina Nozarian,NaN,0.528333,0.525000,3.0,0.032198
9,_Ali Kalantari,NaN,_Alireza Najafi,0.250303,0.250000,11.0,-2.139185


# Enabler

In [23]:
dp2['Best_Match'].value_counts()

_Mohsen Mirzaei        6
_Milad                 5
_Sina Nozarian         3
_Sina Eslami           3
_Mehdi Saadati         2
_Amin                  1
_Payam Zh. Monfared    1
_Alireza Najafi        1
_Hadi                  1
Name: Best_Match, dtype: int64

# Disabler

In [24]:
dp2['Worst_Match'].value_counts()

_Ali Asadinia          4
_Behnam Hajimosaei     3
_Alireza Najafi        3
_Payam Zh. Monfared    2
_Ali Izadi             2
_Ali Kalantari         2
_Alireza Ranjbar       2
_Pedram Ensandoost     1
_Amin                  1
_Sina Nozarian         1
_Ashkan                1
_Afshin                1
Name: Worst_Match, dtype: int64

# Worst Opponents

In [29]:
df1

,Match,Result,Playerno,_Afshin,_Ali Amiri,_Ali Asadinia,_Ali Izadi,_Ali Kalantari,_Alireza Najafi,_Alireza Nazeri,...,_Mehdi Saadati,_Milad,_Mohammad_Amin Ghiasi,_Mohsen Mirzaei,_Payam Zh. Monfared,_Pedram Ensandoost,_Shayan,_Sina Eslami,_Sina Nozarian,_Yousef Nikzad
0,1,0.571429,1.000000,-1.0,0.0,-1.0,0.0,0.0,1.0,0.0,...,1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
1,2,0.500000,1.000000,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0,0.0
2,3,0.550000,1.200000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,-1.0,1.0,1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,1.0
3,4,0.550000,1.000000,-1.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,5,0.600000,1.200000,-1.0,0.0,1.0,-1.0,-1.0,1.0,-1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0.585000,1.000000,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,...,0.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0
96,97,0.475000,1.000000,0.0,1.0,0.0,0.0,-1.0,-1.0,0.0,...,0.0,1.0,1.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,1.0
97,98,0.000000,1.000000,-1.0,1.0,-1.0,0.0,1.0,1.0,1.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0
98,99,0.500000,0.833333,1.0,1.0,1.0,1.0,0.0,-1.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.0


# Splitz:

In [69]:
x_train, x_test, y_train, y_test = train_test_split(df1.drop(columns=['Result','Match']).values, df1['Result'].values, test_size=0.3, random_state=42)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

# x_train,y_train=df1.drop(columns=['Result','Match']).values, df1['Result'].values
# x_test,y_test=x_train,y_train

(70, 24)
(70,)
(30, 24)
(30,)


#### For All data:

In [16]:
# x_train=df1.drop(columns=['Result','Match']).values
# y_train=df1['Result'].values
# print(x_train.shape)
# print(y_train.shape)

# Modeling

# Deep

# Create model

In [26]:
class SingleAttention(Layer):
    def __init__(self, d_k, d_v):
        super(SingleAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v

    def build(self, input_shape):
        self.query = Dense(self.d_k, 
                           input_shape=input_shape, 
#                            kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2),
#                            activation='relu',
                           kernel_initializer='glorot_uniform', 
                           bias_initializer='glorot_uniform')

        self.key = Dense(self.d_k, 
                         input_shape=input_shape, 
#                          kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2),
#                          activation='relu',
                         kernel_initializer='glorot_uniform', 
                         bias_initializer='glorot_uniform')

        self.value = Dense(self.d_v, 
                           input_shape=input_shape, 
#                            kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2),
#                            activation='relu',
                           kernel_initializer='glorot_uniform', 
                           bias_initializer='glorot_uniform')

    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
        q = self.query(inputs[0])
        k = self.key(inputs[1])

        attn_weights = tf.matmul(q, k, transpose_b=True)
        attn_weights = tf.map_fn(lambda x: x/np.sqrt(self.d_k), attn_weights)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)

        v = self.value(inputs[2])
        attn_out = tf.matmul(attn_weights, v)
        return attn_out    
    
    def get_config(self): # Needed for saving and loading model with custom layer
        config = super().get_config().copy()
        config.update({'d_k': self.d_k,
                       'd_v': self.d_v,
                      })
        return config     

## Normal Model

In [27]:
def create_model_normal(lr1):
#     attn_layer1 = SingleAttention(8,8)
#     normalizer=LayerNormalization(input_shape=(8,8), epsilon=1e-6)
#     attn_layer2 = SingleAttention(26,26)
    '''Construct Model 1'''
    in_seq = Input(shape=(x_train.shape[1]))#seq_len
    x=in_seq
    x = Dropout(0.3)(x)
    x = Dense(8, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=3), bias_constraint=max_norm(max_value=3))(x)
    x = Dropout(0.3)(x)
    x = Dense(16, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=3), bias_constraint=max_norm(max_value=3))(x)
    x = Dropout(0.3)(x)
    x = Dense(8, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=3), bias_constraint=max_norm(max_value=3))(x)
    x = Dropout(0.4)(x)
#     x = Dense(11, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=1), bias_constraint=max_norm(max_value=1))(x)
#     x = Dropout(0.4)(x)
    out = Dense(1,activation='sigmoid')(x)

    
    model = Model(inputs=in_seq, outputs=out)
#     model.compile(loss='CategoricalCrossentropy', optimizer=keras.optimizers.Adam(lr=0.00001), metrics=['mae', 'mape'])
    optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
#     optimizer=keras.optimizers.SGD(lr=lr1)
    model.compile(loss= keras.losses.BinaryCrossentropy(), 
                  optimizer=optimizer)

    return model


model1 = create_model_normal(0.0001)
# model2.load_weights('soccer2_Attention.h5')
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24)]              0         
                                                                 
 dropout (Dropout)           (None, 24)                0         
                                                                 
 dense (Dense)               (None, 8)                 200       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 136   

/anaconda/envs/azureml_py38_tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


## Attention Model

In [70]:
def create_model(lr1):
    attn_layer1 = SingleAttention(14,14)
    attn_layer2 = SingleAttention(14,14)
    normalizer=LayerNormalization(input_shape=(8,8), epsilon=1e-6)
    
    '''Construct Model 1'''
    in_seq = Input(shape=(x_train.shape[1]))#seq_len
    x=in_seq
    x = Dense(14, activation='tanh',kernel_initializer='glorot_uniform')(x)
#     x = Dropout(0.2)(x)
#     x = Dense(14, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2))(x)
    
    x1 =x
    x = Dropout(0.3)(x)
    x = attn_layer1((x,x,x))
    x= normalizer(x+x1)  
    x=x+x1
    x1 =x
    x = Dropout(0.3)(x)
    x = attn_layer2((x,x,x))
    x= normalizer(x+x1)
    x=x+x1
    x = Dropout(0.3)(x)
    x = Dense(8, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2))(x)
    x = Dropout(0.4)(x)
    out = Dense(1,activation='sigmoid')(x)

    
    model = Model(inputs=in_seq, outputs=out)
#     model.compile(loss='CategoricalCrossentropy', optimizer=keras.optimizers.Adam(lr=0.00001), metrics=['mae', 'mape'])
    optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
#     optimizer=keras.optimizers.SGD(lr=lr1)
    model.compile(loss= keras.losses.BinaryCrossentropy(), 
                  optimizer=optimizer)

    return model


model2 = create_model(0.0001)
# model2.load_weights('soccer2_Attention_dimensionR_2.h5')
model2.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 dense_15 (Dense)               (None, 14)           350         ['input_9[0][0]']                
                                                                                                  
 dropout_18 (Dropout)           (None, 14)           0           ['dense_15[0][0]']               
                                                                                                  
 single_attention_4 (SingleAtte  (None, 14)          630         ['dropout_18[0][0]',             
 ntion)                                                           'dropout_18[0][0]',       

In [33]:
# lr1=0.001
# optimizer=keras.optimizers.SGD(learning_rate=lr1)
# model.compile(loss= keras.losses.BinaryCrossentropy(), 
#               optimizer=optimizer)
# h=model.fit(x_train,y_train,epochs=5000,batch_size=2,shuffle=True,)

# Encode

In [71]:
def create_model_encode(lr1):
    '''Construct Model 1'''
    in_seq = Input(shape=(x_train.shape[1]))#seq_len
    x=in_seq
#     x = Dropout(0.2)(x)
    x = Dense(14,activation='tanh',kernel_initializer='glorot_uniform',name='Encoded')(x)#,kernel_constraint=max_norm(max_value=3), bias_constraint=max_norm(max_value=3)
#     x = Dropout(0.2)(x)
    out = Dense(x_train.shape[1],activation='tanh')(x)
    model = Model(inputs=in_seq, outputs=out)
#     model.compile(loss='CategoricalCrossentropy', optimizer=keras.optimizers.Adam(lr=0.00001), metrics=['mae', 'mape'])
    optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
#     optimizer=keras.optimizers.SGD(lr=lr1)
    model.compile(loss= keras.losses.MeanAbsoluteError(), #.MeanAbsoluteError(),
                  optimizer=optimizer)

    return model


model3 = create_model_encode(0.001)
model3.summary()
# model3.fit(x_train,x_train,epochs=1000,batch_size=16,shuffle=True)
# model3.save('Encode.h5',include_optimizer=False)
# model3.load_weights('Encode.h5')
# # Weights(Untrainable)
def w_load(layer1,layer2,train_layer=True):
    if len(layer1.get_weights())>0:
        layer1.set_weights(layer2.get_weights())
        if train_layer==False:
            layer1.trainable=False



Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 24)]              0         
                                                                 
 Encoded (Dense)             (None, 14)                350       
                                                                 
 dense_18 (Dense)            (None, 24)                360       
                                                                 
Total params: 710
Trainable params: 710
Non-trainable params: 0
_________________________________________________________________


In [52]:
print(x_train[1])
model3.predict(np.reshape(x_train[1],(1,x_train.shape[1])))

[ 1. -1.  0.  1.  1.  0.  0.  0.  1.  0. -1.  0. -1.  0. -1.  0.  0. -1.
  1.  0.  0. -1.  1.  0.]


array([[ 9.9047858e-01, -8.8729960e-01, -9.6560991e-01,  9.3759155e-01,
         9.6784890e-01, -1.0349667e-03,  1.9886240e-02, -4.0361473e-01,
         9.9408752e-01, -5.1352205e-03, -8.4217566e-01, -1.2524995e-02,
        -9.4824040e-01, -5.2348658e-04, -9.9347502e-01,  1.5774984e-02,
        -4.4033802e-03, -8.9656603e-01,  9.9954361e-01,  5.3525455e-03,
         2.4992153e-03, -9.9989849e-01,  9.3055445e-01, -1.2307610e-02]],
      dtype=float32)

## Train

#### Encode

In [74]:
optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=0.0001)
# optimizer=keras.optimizers.Adam(lr=lr1)
# optimizer=keras.optimizers.SGD(learning_rate=lr1)
model1.compile(loss= keras.losses.MeanAbsoluteError(), 
              optimizer=optimizer)
model3.fit(x_train,x_train,epochs=50,batch_size=16,shuffle=True)
# model3.save('Encode.h5',include_optimizer=False)

Epoch 1/50
5/5 [==============================] - 0s 2ms/step - loss: 0.0254
Epoch 2/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0255
Epoch 3/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0259
Epoch 4/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0256
Epoch 5/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0255
Epoch 6/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0261
Epoch 7/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0262
Epoch 8/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0256
Epoch 9/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0255
Epoch 10/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0252
Epoch 11/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0252
Epoch 12/50
5/5 [==============================] - 0s 1ms/step - loss: 0.0257
Epoch 13/50
5/5 [==============================] - 0s 1ms/step - loss: 0.

#### Model 1

In [342]:
w_load(model1.layers[2],model3.layers[1],train_layer=False)   
lr1=0.001
optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
# optimizer=keras.optimizers.Adam(lr=lr1)
# optimizer=keras.optimizers.SGD(learning_rate=lr1)
model1.compile(loss= keras.losses.BinaryCrossentropy(), 
              optimizer=optimizer)
model1.summary()
# model1.load_weights('soccer2_normal.h5')
with tf.device('/cpu:0'):
    
    h=model1.fit(x_train,y_train,epochs=1000,batch_size=16,shuffle=True)
    model1.save('soccer2_normal_2.h5',include_optimizer=False)
    


Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 28)]              0         
                                                                 
 dropout_82 (Dropout)        (None, 28)                0         
                                                                 
 dense_64 (Dense)            (None, 8)                 232       
                                                                 
 dropout_83 (Dropout)        (None, 8)                 0         
                                                                 
 dense_65 (Dense)            (None, 16)                144       
                                                                 
 dropout_84 (Dropout)        (None, 16)                0         
                                                                 
 dense_66 (Dense)            (None, 8)                 136

#### Model 2

In [75]:
w_load(model2.layers[1],model3.layers[1],train_layer=False)    
lr1=0.0001
optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
# optimizer=keras.optimizers.Adam(lr=lr1)
# optimizer=keras.optimizers.SGD(learning_rate=lr1)
model2.compile(loss= keras.losses.BinaryCrossentropy(), 
              optimizer=optimizer)
# model2.load_weights('soccer2_Attention_dimensionR_2.h5')
# with tf.device('/cpu:0'):
    
#     h=model2.fit(x_train,y_train,epochs=50,batch_size=16,shuffle=True)
#     model2.save('soccer2_Attention_dimensionR_2.h5',include_optimizer=False)

In [76]:
model2.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 dense_15 (Dense)               (None, 14)           350         ['input_9[0][0]']                
                                                                                                  
 dropout_18 (Dropout)           (None, 14)           0           ['dense_15[0][0]']               
                                                                                                  
 single_attention_4 (SingleAtte  (None, 14)          630         ['dropout_18[0][0]',             
 ntion)                                                           'dropout_18[0][0]',       

In [ ]:
with tf.device('/cpu:0'):
    model2.fit(x_train,y_train,epochs=10000,batch_size=32,shuffle=True)

Epoch 1/10000
3/3 [==============================] - 2s 7ms/step - loss: 0.8310
Epoch 2/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.8206
Epoch 3/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.8730
Epoch 4/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.9078
Epoch 5/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.8617
Epoch 6/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.8739
Epoch 7/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.8223
Epoch 8/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.8860
Epoch 9/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.8374
Epoch 10/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.8158
Epoch 11/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.7625
Epoch 12/10000
3/3 [==============================] - 0s 5ms/step - loss: 0.8878
Epoch 13/10000
3/3 [=================

In [67]:
model2.evaluate(x_test,y_test)

4/4 [==============================] - 0s 3ms/step - loss: 0.6302


0.6301590800285339

In [68]:
model2.evaluate(x_train,y_train)

4/4 [==============================] - 0s 3ms/step - loss: 0.6302


0.6301590800285339

# Heavy Model

In [258]:
def create_model(lr1):
    attn_layer1 = SingleAttention(8,8)
    normalizer=LayerNormalization(input_shape=(8,8), epsilon=1e-6)
#     attn_layer2 = SingleAttention(26,26)
    '''Construct Model 1'''
    in_seq = Input(shape=(x_train.shape[1]))#seq_len
    x=in_seq
    x = Dropout(0.2)(x)
    x = Dense(8, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2))(x)
    x1 =x
    x = Dropout(0.2)(x)
    x = attn_layer1((x,x,x))
    x= normalizer(x+x1)
    
    x = Dense(8, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=2), bias_constraint=max_norm(max_value=2))(x)
    x1 =x
    x = Dropout(0.2)(x)
    x = attn_layer1((x,x,x))
    x= normalizer(x+x1)
    
    x = Dropout(0.3)(x)
    x = Dense(16, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=3), bias_constraint=max_norm(max_value=3))(x)
    x = Dropout(0.4)(x)
#     x = Dense(11, activation='tanh',kernel_initializer='glorot_uniform',kernel_constraint=max_norm(max_value=1), bias_constraint=max_norm(max_value=1))(x)
#     x = Dropout(0.4)(x)
    out = Dense(1,activation='sigmoid')(x)

    
    model = Model(inputs=in_seq, outputs=out)
#     model.compile(loss='CategoricalCrossentropy', optimizer=keras.optimizers.Adam(lr=0.00001), metrics=['mae', 'mape'])
    optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
#     optimizer=keras.optimizers.SGD(lr=lr1)
    model.compile(loss= keras.losses.BinaryCrossentropy(), 
                  optimizer=optimizer)

    return model


model2 = create_model(0.0001)
# model2.load_weights('soccer2_Attention.h5')
model2.summary()

Model: "model_74"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 28)]         0           []                               
                                                                                                  
 dropout_39 (Dropout)           (None, 28)           0           ['input_20[0][0]']               
                                                                                                  
 dense_29 (Dense)               (None, 8)            232         ['dropout_39[0][0]']             
                                                                                                  
 dropout_40 (Dropout)           (None, 8)            0           ['dense_29[0][0]']               
                                                                                           

/anaconda/envs/azureml_py38_tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


In [ ]:
w_load(model2.layers[2],model3.layers[2],train_layer=False) 
model2.summary()
lr1=0.0001
optimizer=keras.optimizers.Nadam(beta_1=0.9,beta_2=0.999,lr=lr1)
# optimizer=keras.optimizers.Adam(lr=lr1)
# optimizer=keras.optimizers.SGD(learning_rate=lr1)
model2.compile(loss= keras.losses.BinaryCrossentropy(), 
              optimizer=optimizer)
# model2.load_weights('soccer2_Attention_dimensionR.h5')
with tf.device('/cpu:0'):
    
    h=model2.fit(x_train,y_train,epochs=10000,batch_size=16,shuffle=True)
#     model2.save('soccer2_Attention_dimensionR.h5',include_optimizer=False)

Model: "model_74"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 28)]         0           []                               
                                                                                                  
 dropout_39 (Dropout)           (None, 28)           0           ['input_20[0][0]']               
                                                                                                  
 dense_29 (Dense)               (None, 8)            232         ['dropout_39[0][0]']             
                                                                                                  
 dropout_40 (Dropout)           (None, 8)            0           ['dense_29[0][0]']               
                                                                                           

/anaconda/envs/azureml_py38_tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/nadam.py:73: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Nadam, self).__init__(name, **kwargs)


6/6 [==============================] - 2s 4ms/step - loss: 0.6723
Epoch 2/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6795
Epoch 3/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6812
Epoch 4/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6950
Epoch 5/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6891
Epoch 6/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6970
Epoch 7/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6860
Epoch 8/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6804
Epoch 9/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6951
Epoch 10/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6902
Epoch 11/10000
6/6 [==============================] - 0s 3ms/step - loss: 0.6925
Epoch 12/10000
6/6 [==============================] - 0s 4ms/step - loss: 0.6915
Epoch 13/10000
6/6 [==============================]

## Retrain:

In [31]:
# with tf.device("/cpu:1"):
#     model2=keras.models.load_model('soccer2_Attention_dimensionR.h5',compile=False)

#### Train last xx matches

In [74]:
def train_latest_data(x_train,y_train,model,latest=[1],lr1=0.0001,epoch=1000):
#Picking 2 records of latest match:
    indexes_1=[]
    indexes_2=[]
    for j in latest:
        indexes_1.append(len(x_train)//2-j)
        indexes_2.append(len(x_train)-j)
    all_indexes=indexes_1
    all_indexes.extend(indexes_2)
    
    x_train_n=x_train[all_indexes]
    y_train_n=y_train[all_indexes]

    optimizer=keras.optimizers.SGD(lr=lr1,momentum=0.2)
    model.compile(loss= keras.losses.BinaryCrossentropy(), 
                  optimizer=optimizer)
    h=model.fit(x_train_n,y_train_n,epochs=epoch,batch_size=16,shuffle=True)
    
# for i in range(1,6):
train_latest_data(x_train,y_train,model2,latest=[i for i in range(1,15)],epoch=700)

Epoch 1/700


/anaconda/envs/azureml_py38_tensorflow/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


2/2 [==============================] - 2s 8ms/step - loss: 0.5633
Epoch 2/700
2/2 [==============================] - 0s 5ms/step - loss: 0.5182
Epoch 3/700
2/2 [==============================] - 0s 5ms/step - loss: 0.5737
Epoch 4/700
2/2 [==============================] - 0s 4ms/step - loss: 0.5372
Epoch 5/700
2/2 [==============================] - 0s 5ms/step - loss: 0.5371
Epoch 6/700
2/2 [==============================] - 0s 4ms/step - loss: 0.5466
Epoch 7/700
2/2 [==============================] - 0s 5ms/step - loss: 0.5335
Epoch 8/700
2/2 [==============================] - 0s 5ms/step - loss: 0.5524
Epoch 9/700
2/2 [==============================] - 0s 5ms/step - loss: 0.5326
Epoch 10/700
2/2 [==============================] - 0s 4ms/step - loss: 0.5327
Epoch 11/700
2/2 [==============================] - 0s 4ms/step - loss: 0.5581
Epoch 12/700
2/2 [==============================] - 0s 4ms/step - loss: 0.5080
Epoch 13/700
2/2 [==============================] - 0s 4ms/step - loss: 0

# Saves

In [364]:
model2.save('soccer2_Attention_dimensionR_2.h5',include_optimizer=False)

In [397]:
model.save('soccer2.h5',include_optimizer=False)

# Monitoring results

#### Model 1

In [21]:
p=model1.predict(x_test)
dr=pd.DataFrame()
dr['Predicted']=np.reshape(p,p.shape[0])
dr['Real']=y_test
diff=(abs(dr['Real']-dr['Predicted'])*100)**2
print(sum(diff[-25:]))
dr

23848.717449023585


,Predicted,Real
0,0.502735,0.571429
1,0.521474,0.500000
2,0.580580,0.550000
3,0.514563,0.550000
4,0.588948,0.600000
...,...,...
91,0.473807,0.450000
92,0.548262,0.300000
93,0.568532,0.585000
94,0.565424,0.475000


In [23]:
p=model2.predict(x_test)
dr=pd.DataFrame()
dr['Predicted']=np.reshape(p,p.shape[0])
dr['Real']=y_test
diff=(abs(dr['Real']-dr['Predicted'])*100)**2
print(sum(diff[-25:]))
dr

5315.140312796402


,Predicted,Real
0,0.446577,0.571429
1,0.389717,0.500000
2,0.492088,0.550000
3,0.746425,0.550000
4,0.549769,0.600000
...,...,...
91,0.591002,0.450000
92,0.348360,0.300000
93,0.469171,0.585000
94,0.621287,0.475000


In [22]:
p=model2.predict(x_test)
dr=pd.DataFrame()
dr['Predicted']=np.reshape(p,p.shape[0])
dr['Real']=y_test
diff=(abs(dr['Real']-dr['Predicted'])*100)**2
print(sum(diff[-25:]))
dr

5315.140312796402


,Predicted,Real
0,0.446577,0.571429
1,0.389717,0.500000
2,0.492088,0.550000
3,0.746425,0.550000
4,0.549769,0.600000
...,...,...
91,0.591002,0.450000
92,0.348360,0.300000
93,0.469171,0.585000
94,0.621287,0.475000


# classic models

In [41]:
# df1['Result']=df1['Result']-0.5

In [24]:
regr = RandomForestRegressor(max_depth=5, random_state=0,n_estimators=15)
regr.fit(df1.drop(columns=['Result','Match']).values[:-2], df1['Result'][:-2])
# print(regr.predict([[0, 0, 0, 0]]))

RandomForestRegressor(max_depth=5, n_estimators=15, random_state=0)

In [25]:
reg = LinearRegression().fit(df1.drop(columns=['Result','Match']).values, df1['Result'])

In [26]:
p2=regr.predict(x_test)
dr2=pd.DataFrame()
dr2['Predicted']=p2
dr2['Real']=y_test
diff=((dr2['Real']-dr2['Predicted'])*100)**2
print(sum(diff[-25:]))
dr2

9070.982107634967


,Predicted,Real
0,0.531515,0.571429
1,0.447830,0.500000
2,0.534588,0.550000
3,0.547875,0.550000
4,0.566604,0.600000
...,...,...
91,0.552250,0.450000
92,0.434810,0.300000
93,0.564946,0.585000
94,0.604521,0.475000


# Importance

In [27]:
rate=dict(zip(df1.columns[2:],regr.feature_importances_))
dict(sorted(rate.items(), key=lambda item: item[1]))

# rate=dict(zip(df1.columns[2:],reg.coef_))
# dict(sorted(rate.items(), key=lambda item: item[1]))

{'_Mohammad_Amin Ghiasi': 0.0019204019423866433,
 '_Jamshid': 0.001989032786617183,
 '_Saeid Hobbi': 0.003432648990813437,
 '_Yousef Nikzad': 0.006973276064077519,
 '_Ali Asadinia': 0.009212666361135595,
 '_Ali Amiri': 0.010602770794892658,
 '_Shayan': 0.011373962826623561,
 '_Alireza Najafi': 0.011426754809807009,
 '_Hadi': 0.012298757193839356,
 '_Amin': 0.012783165785921459,
 '_Alireza Nazeri': 0.014219032354554904,
 '_Sina Eslami': 0.01636787292209747,
 '_Payam Zh. Monfared': 0.017089632309167237,
 '_Alireza Ranjbar': 0.017897553911689526,
 '_Ali Izadi': 0.018196501642370924,
 '_Behnam Hajimosaei': 0.02488254668176125,
 '_Erfan Moeini': 0.029383934232629985,
 '_Saleh Mohammad Rezaei': 0.031162747463741248,
 'Playerno': 0.031879752286772,
 '_Ashkan': 0.036106471703649834,
 '_Mehdi Saadati': 0.03903059332852818,
 '_Ali Kalantari': 0.040093925664336554,
 '_Pedram Ensandoost': 0.04199896365444419,
 '_Sina Nozarian': 0.04223371272663442,
 '_Afshin': 0.053120078466329934,
 '_Milad': 0.05

# Mean win

In [28]:
topz=[]
dic1={}
for i in df1.columns[3:]:
    dic1[i]=df1[df1[i]==1]['Result'].mean()-0.5

In [29]:
Rating=dict(sorted(dic1.items(), key=lambda item: item[1]))

In [30]:
Rating

{'_Saleh Mohammad Rezaei': -0.25,
 '_Ali Kalantari': -0.14077777776666667,
 '_Behnam Hajimosaei': -0.08758095238000002,
 '_Alireza Nazeri': -0.06469187676470584,
 '_Alireza Najafi': -0.040661375652777776,
 '_Ali Asadinia': -0.039847883597222244,
 '_Yousef Nikzad': -0.03225,
 '_Ali Izadi': -0.023333333328124983,
 '_Payam Zh. Monfared': -0.018515873016666684,
 '_Alireza Ranjbar': -0.018083333337500007,
 '_Ashkan': -0.01749999999999996,
 '_Afshin': -0.008809523814814813,
 '_Mohammad_Amin Ghiasi': -0.005568783055555526,
 '_Hadi': -0.005089285718750003,
 '_Jamshid': 0.009642857150000017,
 '_Amin': 0.010714285714285676,
 '_Erfan Moeini': 0.01666666666666672,
 '_Sina Nozarian': 0.020264550263888848,
 '_Milad': 0.037056277045454555,
 '_Sina Eslami': 0.06788177339655166,
 '_Mehdi Saadati': 0.07277777777777772,
 '_Pedram Ensandoost': 0.08125000000000004,
 '_Shayan': 0.0842857142857143,
 '_Ali Amiri': 0.08694805195454547,
 '_Saeid Hobbi': 0.10733333329999994,
 '_Maghsad': 0.16666666666666663,
 '_

# Pearson Correlation

In [31]:
feat=df1.corr('pearson')[['Result']]

feat['Result']=abs(feat['Result'])
feat[2:].sort_values('Result',ascending=False)

,Result
_Mohsen Mirzaei,0.440466
_Saleh Mohammad Rezaei,0.284562
_Ali Kalantari,0.277544
_Behnam Hajimosaei,0.222911
_Mehdi Saadati,0.192501
_Sina Eslami,0.189446
Playerno,0.168384
_Pedram Ensandoost,0.165438
_Maghsad,0.146947
_Ali Amiri,0.146794


# Pipeline

In [32]:
names=df1.columns[3:]
names

Index(['_Afshin', '_Ali Amiri', '_Ali Asadinia', '_Ali Izadi',
       '_Ali Kalantari', '_Alireza Najafi', '_Alireza Nazeri',
       '_Alireza Ranjbar', '_Amin', '_Ashkan', '_Behnam Hajimosaei',
       '_Erfan Moeini', '_Hadi', '_Jamshid', '_Maghsad', '_Mehdi Saadati',
       '_Milad', '_Mohammad_Amin Ghiasi', '_Mohsen Mirzaei',
       '_Payam Zh. Monfared', '_Pedram Ensandoost', '_Saeid Hobbi',
       '_Saleh Mohammad Rezaei', '_Shayan', '_Sina Eslami', '_Sina Nozarian',
       '_Yousef Nikzad'],
      dtype='object')

### Load Model

In [28]:
model=keras.models.load_model('soccer2.h5',compile=False)

In [29]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 26)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 15)                405       
_________________________________________________________________
dropout_5 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 16        
Total params: 421
Trainable params: 421
Non-trainable params: 0
_________________________________________________________________


In [33]:
names

Index(['_Afshin', '_Ali Amiri', '_Ali Asadinia', '_Ali Izadi',
       '_Ali Kalantari', '_Alireza Najafi', '_Alireza Nazeri',
       '_Alireza Ranjbar', '_Amin', '_Ashkan', '_Behnam Hajimosaei',
       '_Erfan Moeini', '_Hadi', '_Jamshid', '_Maghsad', '_Mehdi Saadati',
       '_Milad', '_Mohammad_Amin Ghiasi', '_Mohsen Mirzaei',
       '_Payam Zh. Monfared', '_Pedram Ensandoost', '_Saeid Hobbi',
       '_Saleh Mohammad Rezaei', '_Shayan', '_Sina Eslami', '_Sina Nozarian',
       '_Yousef Nikzad'],
      dtype='object')

In [34]:
team1=['_Pedram Ensandoost','_Sina Eslami','_Payam Zh. Monfared','_Yousef Nikzad','_Ali Kalantari','_Jamshid']#,'_Behnam Hajimosaei'
team2=['_Sina Nozarian','_Ali Asadinia','_Alireza Ranjbar','_Hadi','_Mohammad_Amin Ghiasi','_Alireza Najafi']#
,
print(len(team1))
print(len(team2))

6
6


In [35]:
def get_x_ready(team1,team2,has__=False):
    if has__==False:
        teams1=['_'+i for i in team1]
        teams2=['_'+i for i in team2]
    else:
        teams1=team1
        teams2=team2
#     teams=teams1
#     teams.extend(teams2)
#     teams=np.array(teams)
    dt=pd.DataFrame(columns=names)
    dt.loc[0]=np.zeros(len(names))#All zero
    dt[teams1]=1
    dt[teams2]=-1
    return dt.values

In [36]:
def predict_game(model,regr,team1,team2,has__=False,print_stuff=True):
#     with tf.device('/cpu:0'):
    teamz1=team1
    teamz2=team2
    playerno=len(teamz1)/len(teamz2)
    x1=np.expand_dims(np.append(playerno,get_x_ready(teamz1,teamz2,has__)),0)
    
    #Predict,swap,predict
    rez1=model.predict(x1)[0][0]
    rez2=regr.predict(x1)[0]
    
    #Swap
    x2=np.expand_dims(np.append(1/playerno,get_x_ready(teamz2,teamz1,has__)),0)

    rez11=1-model.predict(x2)[0][0]#1-
    rez22=1-(regr.predict(x2)[0])    
    rez_final=(rez1+rez11)/2
    rez1=round((rez1+rez11)/2,2)
    rez2=round((rez2+rez22)/2,2)
    if print_stuff:
        #Deep:
        if rez1>0.55:
            print(f'Deep model prediction: Team 1 will Win with a probability of: {rez1}')
        elif rez1<0.45:
            print(f'Deep model prediction: Team 2 will Win with a probability of: {1-rez1}')
        else:
            print(f"Deep model prediction: Likely Draw, with {rez1} chance of Team 1 winning")

        #Classic:
        if rez2>0.55:
            print(f'Classic model prediction: Team 1 will Win with a probability of: {rez2}')
        elif rez2<0.45:
            print(f'Classic model prediction: Team 2 will Win with a probability of: {1-rez2}')
        else:
            print(f"Classic model prediction: Likely Draw, with {rez2} chance of Team 1 winning")    
    
    return rez_final
    

In [37]:
predict_game(model2,regr,team1,team2,has__=True)

Deep model prediction: Team 1 will Win with a probability of: 0.61
Classic model prediction: Team 2 will Win with a probability of: 0.56


0.6086461842060089

# Team players recommendation

In [75]:
names

Index(['_Afshin', '_Ali Amiri', '_Ali Asadinia', '_Ali Izadi',
       '_Ali Kalantari', '_Alireza Najafi', '_Alireza Nazeri',
       '_Alireza Ranjbar', '_Amin', '_Ashkan', '_Behnam Hajimosaei',
       '_Erfan Moeini', '_Hadi', '_Jamshid', '_Mehdi Saadati', '_Milad',
       '_Mohammad_Amin Ghiasi', '_Mohsen Mirzaei', '_Payam Zh. Monfared',
       '_Pedram Ensandoost', '_Saeid Hobbi', '_Saleh Mohammad Rezaei',
       '_Sina Eslami', '_Sina Nozarian', '_Yousef Nikzad'],
      dtype='object')

In [143]:
#List of All players except keepers:
team1=['_Alireza Nazeri','_Sina Eslami','_Mehdi Saadati','_Afshin','_Ali Izadi']#,'_Mohsen Mirzaei'
team2=['_Payam Zh. Monfared','_Alireza Ranjbar','_Hadi','_Alireza Najafi']#'_Milad','_Ali Asadinia'

all_team=team1
all_team.extend(team2)
all_team
#['_Milad','_Yousef Nikzad','_Ali Asadinia','_Alireza Ranjbar','_Alireza Najafi','_Payam Zh. Monfared','_Pedram Ensandoost','_Amin','_Mohsen Mirzaei','_Mohammad_Amin Ghiasi','_Afshin','_Ali Izadi']

['_Alireza Nazeri',
 '_Sina Eslami',
 '_Mehdi Saadati',
 '_Afshin',
 '_Ali Izadi',
 '_Payam Zh. Monfared',
 '_Alireza Ranjbar',
 '_Hadi',
 '_Alireza Najafi']

### We make 2 lists of players, randomly append players to them, then calculate the win probability

In [399]:
triez=200
formation=[]
numbz=[]
rezultz=[]
h=len(all_team)//2 #Half

for i in range (triez):
    teamz=all_team
    #Setting keepers aside:
    t1=['_Amin','_Mohsen Mirzaei']#['_Sina Nozarian']
    t2=['_Pedram Ensandoost','_Milad','_Ali Asadinia']#['_Behnam Hajimosaei']
    
    #Shuffle & Pick:
    np.random.seed(i)
    np.random.shuffle(teamz)
    
    if teamz not in formation: #If this formation hasn't been done before:
        teamz1=teamz.copy() #This is a must, or python gets stupid
        formation.append(teamz1)
        t1.extend(teamz[:h])
        t2.extend(teamz[h:])
        result=predict_game(model,regr,t1,t2,has__=True,print_stuff=False)
        rezultz.append(result)

### Now we pick the top 5 ones that are closest to 50%

Problem: Duplicate teams. we use reverse dictionaries to remove duplicate values

In [400]:
def reverse_dict(rezultz_dict):
    return dict(zip(rezultz_dict.values(),rezultz_dict.keys()))

In [401]:
#For Indexing, we make a dictionary for formation and results
formation_dict=dict(zip([i for i in range (triez)],formation))
rezultz_dict=dict(zip([i for i in range (triez)],abs(np.array(rezultz)-0.5)))

print(len(rezultz_dict))
rezultz_dict1=reverse_dict(reverse_dict(rezultz_dict))
print(len(rezultz_dict1))

top10=list(dict(sorted(rezultz_dict1.items(), key=lambda item: item[1])).keys())[:10]

200
101


In [402]:
# If constant ones werent just keepers
h=5

In [403]:
team_frame=pd.DataFrame()
for c,i in enumerate(top10):
    t1=['_Amin','_Mohsen Mirzaei']#['_Sina Nozarian']
    t2=['_Pedram Ensandoost','_Milad','_Ali Asadinia']#['_Behnam Hajimosaei']
    t1.extend(formation_dict[i][:h])
    t2.extend(formation_dict[i][h:])
    result_1=float(predict_game(model,regr,t1,t2,has__=True,print_stuff=False))
    result_2=float(predict_game(model,regr,t2,t1,has__=True,print_stuff=False))
    t1.append(result_1)
    t2.append(result_2)
    team_frame[f'TeamA-{c+1}']=t1
    team_frame[f'TeamB-{c+1}']=t2
team_frame

,TeamA-1,TeamB-1,TeamA-2,TeamB-2,TeamA-3,TeamB-3,TeamA-4,TeamB-4,TeamA-5,TeamB-5,TeamA-6,TeamB-6,TeamA-7,TeamB-7,TeamA-8,TeamB-8,TeamA-9,TeamB-9,TeamA-10,TeamB-10
0,_Amin,_Pedram Ensandoost,_Amin,_Pedram Ensandoost,_Amin,_Pedram Ensandoost,_Amin,_Pedram Ensandoost,_Amin,_Pedram Ensandoost,_Amin,_Pedram Ensandoost,_Amin,_Pedram Ensandoost,_Amin,_Pedram Ensandoost,_Amin,_Pedram Ensandoost,_Amin,_Pedram Ensandoost
1,_Mohsen Mirzaei,_Milad,_Mohsen Mirzaei,_Milad,_Mohsen Mirzaei,_Milad,_Mohsen Mirzaei,_Milad,_Mohsen Mirzaei,_Milad,_Mohsen Mirzaei,_Milad,_Mohsen Mirzaei,_Milad,_Mohsen Mirzaei,_Milad,_Mohsen Mirzaei,_Milad,_Mohsen Mirzaei,_Milad
2,_Payam Zh. Monfared,_Ali Asadinia,_Alireza Ranjbar,_Ali Asadinia,_Mehdi Saadati,_Ali Asadinia,_Alireza Nazeri,_Ali Asadinia,_Afshin,_Ali Asadinia,_Mehdi Saadati,_Ali Asadinia,_Ali Izadi,_Ali Asadinia,_Payam Zh. Monfared,_Ali Asadinia,_Payam Zh. Monfared,_Ali Asadinia,_Mehdi Saadati,_Ali Asadinia
3,_Alireza Nazeri,_Ali Izadi,_Mehdi Saadati,_Alireza Najafi,_Payam Zh. Monfared,_Hadi,_Mehdi Saadati,_Payam Zh. Monfared,_Sina Eslami,_Alireza Nazeri,_Sina Eslami,_Afshin,_Mehdi Saadati,_Alireza Najafi,_Afshin,_Hadi,_Mehdi Saadati,_Ali Izadi,_Sina Eslami,_Payam Zh. Monfared
4,_Sina Eslami,_Alireza Ranjbar,_Ali Izadi,_Hadi,_Afshin,_Alireza Najafi,_Sina Eslami,_Hadi,_Mehdi Saadati,_Alireza Najafi,_Ali Izadi,_Alireza Nazeri,_Alireza Nazeri,_Hadi,_Mehdi Saadati,_Sina Eslami,_Alireza Ranjbar,_Afshin,_Alireza Najafi,_Alireza Ranjbar
5,_Mehdi Saadati,_Alireza Najafi,_Payam Zh. Monfared,_Afshin,_Ali Izadi,_Sina Eslami,_Alireza Ranjbar,_Afshin,_Alireza Ranjbar,_Hadi,_Alireza Ranjbar,_Payam Zh. Monfared,_Afshin,_Sina Eslami,_Alireza Ranjbar,_Alireza Najafi,_Sina Eslami,_Alireza Najafi,_Ali Izadi,_Afshin
6,_Afshin,_Hadi,_Sina Eslami,_Alireza Nazeri,_Alireza Nazeri,_Alireza Ranjbar,_Alireza Najafi,_Ali Izadi,_Payam Zh. Monfared,_Ali Izadi,_Hadi,_Alireza Najafi,_Payam Zh. Monfared,_Alireza Ranjbar,_Alireza Nazeri,_Ali Izadi,_Alireza Nazeri,_Hadi,_Hadi,_Alireza Nazeri
7,0.512431,0.487569,0.521994,0.478006,0.501872,0.498128,0.43823,0.56177,0.505161,0.494839,0.404615,0.595385,0.501872,0.498128,0.501887,0.498113,0.488514,0.511486,0.368648,0.631352


In [404]:
dict(sorted(rezultz_dict1.items(), key=lambda item: item[1]))

{72: 0.0004761219024658203,
 98: 0.0042280107736587524,
 118: 0.004918202757835388,
 195: 0.005536332726478577,
 140: 0.008006542921066284,
 167: 0.009554147720336914,
 73: 0.010331302881240845,
 196: 0.015326783061027527,
 49: 0.01668904721736908,
 151: 0.019183874130249023,
 139: 0.020767316222190857,
 105: 0.02340252697467804,
 81: 0.024936899542808533,
 31: 0.025064736604690552,
 189: 0.025600120425224304,
 93: 0.025787681341171265,
 135: 0.0265425443649292,
 156: 0.02902369201183319,
 111: 0.029246225953102112,
 192: 0.0321720689535141,
 161: 0.04085525870323181,
 150: 0.04345810413360596,
 163: 0.052444204688072205,
 193: 0.052794620394706726,
 101: 0.055782392621040344,
 54: 0.05885292589664459,
 146: 0.06028226017951965,
 152: 0.0684533566236496,
 169: 0.07478144764900208,
 143: 0.07571835815906525,
 122: 0.07863534986972809,
 75: 0.08040028810501099,
 141: 0.09407661855220795,
 13: 0.0953093022108078,
 197: 0.09698700904846191,
 147: 0.0977374017238617,
 198: 0.101202830672264